<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"></center><br/>

# Assignment: Notebook for Peer Assignment

Estimated time needed: 60 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Weather forecast firm as a data scientist.

The company is considering the weather condition to help predict the possibility of precipitations, which involves using various local climatological variables, including temperature, wind speed, humidity, dew point, and pressure. The data you will be handling was collected by a NOAA weather station located at the John F. Kennedy International Airport in Queens, New York.

Your task is to provide a high level analysis of weather data in JFK Airport. Your stakeholders want to understand the current and historical record of precipitations based on different variables. For now they are mainly interested in a macro-view of JFK Airport Weather, and how it relates to the possibility to rain because it will affect flight delays and etc.


# Introduction

This project relates to the NOAA Weather Dataset - JFK Airport (New York). The original dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature and wind speed) collected at JFK airport. This dataset can be obtained for free from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0151ENSkillsNetwork926-2022-01-01). 

For this project, you will be using a subset dataset, which contains 5727 rows (about 5% or original rows) and 9 columns. The end goal will be to predict the precipitation using some of the available features. In this project, you will practice reading data files, preprocessing data, creating models, improving models and evaluating them to ultimately choose the best model.




## Table of Contents:

Using this R notebook you will complete **10 tasks**:
* [0. Import Modules](#cell0)
* [1. Download and Unzip NOAA Weather Dataset](#cell1)
* [2. Read Dataset into Project](#cell2)
* [3. Select Subset of Columns](#cell3)
* [4. Clean Up Columns](#cell4)
* [5. Convert Columns to Numerical Types](#cell5)
* [6. Rename Columns](#cell6)
* [7. Exploratory Data Analysis](#cell7)
* [8. Linear Regression](#cell8)
* [9. Improve the Model](#cell9)
* [10. Find Best Model](#cell10)


<a id="cell0"></a>
## 0. Import required modules

Below, install "tidymodels", additionally "rlang" should be updated in order to properly run "tidymodels". 


In [ ]:
# Install tidymodels if you haven't done so
install.packages("rlang")
install.packages("tidymodels")

**Note: After installing the packages, restart the kernel. Without installing the packages again, load them. Tidyverse and Tidymodels will be the two main packages you will use.**


In [ ]:
# Library for modeling
library(tidymodels)

# Load tidyverse
library(tidyverse)


### Understand the Dataset

The original NOAA JFK dataset contains 114,546 hourly observations of various local climatological variables (including temperature, wind speed, humidity, dew point, and pressure). 

In this project you will use a sample dataset, which is around 293 KB. [Link to the sample dataset](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz).

The sample contains 5727 rows (about 5% or original rows) and 9 columns, which are:
- DATE
- HOURLYDewPointTempF
- HOURLYRelativeHumidity
- HOURLYDRYBULBTEMPF
- HOURLYWETBULBTEMPF
- HOURLYPrecip
- HOURLYWindSpeed
- HOURLYSeaLevelPressure
- HOURLYStationPressure

The original dataset is much bigger. Feel free to explore the original dataset. [Link to the original dataset.](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa_weather.html) 

For more information about the dataset, checkout the [preview](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/data-preview/index.html?_ga=2.176781478.281508226.1616293518-1509963377.1616117067&cm_mc_uid=90945889198916153255549&cm_mc_sid_50200000=64650651616293516933) of NOAA Weather - JFK Airport.


<a id="cell1"></a>

## 1. Download NOAA Weather Dataset

Use the `download.file()` function to download the sample dataset from the URL below.

URL = 'https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz'


In [1]:
download.file(url = "https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz", destfile = "noaa-weather-sample.tar.gz")

NameError: name 'download' is not defined

Untar the zipped file.


In [2]:
untar("noaa-weather-sample.tar.gz")

NameError: name 'untar' is not defined

<a id="cell2"></a>
## 2. Extract and Read into Project
We start by reading in the raw dataset. You should specify the file name as "noaa-weather-sample-data/jfk_weather_sample.csv".


In [3]:
noaa_data <- read_csv("jfk_weather_sample.csv")

NameError: name 'noaa_data' is not defined

Next, display the first few rows of the dataframe.


In [4]:
head(noaa_data)

NameError: name 'head' is not defined

Also, take a `glimpse` of the dataset to see the different column data types and make sure it is the correct subset dataset with about 5700 rows and 9 columns.


In [5]:
glimpse(noaa_data)

NameError: name 'glimpse' is not defined

<a id="cell3"></a>
## 3. Select Subset of Columns

The end goal of this project will be to predict `HOURLYprecip` (precipitation) using a few other variables. Before you can do this, you first need to preprocess the dataset. Section 3 to section 6 focuses on preprocessing.

The first step in preprocessing is to select a subset of data columns and inspect the column types.

The key columns that we will explore in this project are:
- HOURLYRelativeHumidity
- HOURLYDRYBULBTEMPF
- HOURLYPrecip
- HOURLYWindSpeed
- HOURLYStationPressure

Data Glossary:
- 'HOURLYRelativeHumidity' is the relative humidity given to the nearest whole percentage.
- 'HOURLYDRYBULBTEMPF' is the dry-bulb temperature and is commonly used as the standard air temperature reported. It is given here in whole degrees Fahrenheit.
- 'HOURLYPrecip' is the amount of precipitation in inches to hundredths over the past hour. For certain automated stations, precipitation will be reported at sub-hourly intervals (e.g. every 15 or 20 minutes) as an accumulated amount of all precipitation within the preceding hour. A “T” indicates a trace amount of precipitation.
- 'HOURLYWindSpeed' is the speed of the wind at the time of observation given in miles per hour (mph).
- 'HOURLYStationPressure' is the atmospheric pressure observed at the station during the time of observation. Given in inches of Mercury (in Hg).

`Select` those five columns and store the modified dataframe as a new variable.


In [6]:
noaa_subset <- select(noaa_data, 'HOURLYRelativeHumidity', 'HOURLYDRYBULBTEMPF' , 'HOURLYPrecip' , 
       'HOURLYWindSpeed' , 'HOURLYStationPressure')

NameError: name 'noaa_subset' is not defined

Show the first 10 rows of this new dataframe.


In [7]:
head(noaa_subset, 10)

NameError: name 'head' is not defined

<a id="cell4"></a>
## 4. Clean Up Columns

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NA` and `T` values. `T` specifies *trace amounts of precipitation* (meaning essentially no precipitation), while `NA` means *not available*, and is used to denote missing values. Additionally, some values also have "s" at the end of them, indicating that the precipitation was snow. 

Inspect the unique values present in the column `HOURLYPrecip` (with `unique(dataframe$column)`) to see these values.


In [8]:
unique(noaa_subset$HOURLYPrecip)

SyntaxError: invalid syntax (2254242018.py, line 1)

Having characters in values (like the "T" and "s" that you see in the unique values) will cause problems when you create a model because values for precipitation should be numerical. So you need to fix these values that have characters. 

Now, for the column `HOURLYPrecip`:
1. Replace all the `T` values with "0.0" and 
2. Remove "s" from values like "0.02s". In R, you can use the method `str_remove(column, pattern = "s$")` to remove the character "s" from the end of values. The "$" tells R to match to the end of values. The `pattern` is a regex pattern. Look at [here](https://www.rdocumentation.org/packages/stringi/versions/1.5.3/topics/about_search_regex?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDA0151ENSkillsNetwork926-2022-01-01) for more information about regex and matching to strings in R.

Remember that you can use `tidyverse`'s  `mutate()` to update columns.

You can check your work by checking if unique values of `HOURLYPrecip` still contain any `T` or `s`. Store the modified dataframe as a new variable.


In [9]:
noaa_subsetwrangled <-  noaa_subset %>% mutate(HOURLYPrecip = ifelse(HOURLYPrecip == "T", "0.0", HOURLYPrecip)) # replacing the "T" in the column HOURLYPrecip with "0.0"
unique(noaa_subsetwrangled$HOURLYPrecip) # checking if the "T"s are gone

noaa_subsetwrangled <- noaa_subsetwrangled %>% mutate(HOURLYPrecip = str_remove(HOURLYPrecip, pattern = "s$")) # removing the "s" in the end of the numbers
unique(noaa_subsetwrangled$HOURLYPrecip) # checking if the "s" are gone too

SyntaxError: invalid syntax (1719062920.py, line 1)

<a id="cell5"></a>
## 5. Convert Columns to Numerical Types
Now that you have removed the characters in the `HOURLYPrecip` column, you can safely covert the column to a numeric type.

First, check the types of the columns. You will notice that all are `dbl` (double or numeric) except for `HOURLYPrecip`, which is `chr` (character or string). Use the `glimpse` function from Tidyverse.


In [10]:
glimpse(noaa_subsetwrangled)

NameError: name 'glimpse' is not defined

Convert `HOURLYPrecip` to the `numeric` type and store the cleaned dataframe as a new variable.


In [11]:
noaa_subsetwrangledn <- noaa_subsetwrangled %>% mutate(HOURLYPrecip = as.numeric(HOURLYPrecip))

SyntaxError: invalid syntax (2229390513.py, line 1)

We can now see that all fields have numerical data type.


In [12]:
glimpse(noaa_subsetwrangledn)

NameError: name 'glimpse' is not defined

<a id="cell6"></a>
## 6. Rename Columns
Let's rename the following columns as:
- 'HOURLYRelativeHumidity' to 'relative_humidity'
- 'HOURLYDRYBULBTEMPF' to 'dry_bulb_temp_f'
- 'HOURLYPrecip' to 'precip'
- 'HOURLYWindSpeed' to 'wind_speed'
- 'HOURLYStationPressure' to 'station_pressure'

You can use `dplyr::rename()`. Then, store the final dataframe as a new variable.


In [13]:
noaa_subsetwranglednn <- noaa_subsetwrangledn # creating a new data frame

colnames(noaa_subsetwranglednn) <- c("relative_humidity", "dry_bulb_temp_f", 
                                     "precip", "wind_speed", "station_pressure") # changing the column names to what was requested

glimpse(noaa_subsetwranglednn)  # checking whether the changes took place

NameError: name 'noaa_subsetwranglednn' is not defined

<a id="cell7"></a>
## 7. Exploratory Data Analysis
Now that you have finished preprocessing the dataset, you can can start exploring the columns more.

First, split the data into a training and testing set. Splitting a dataset is done randomly, so to have reproducible results set the seed = 1234. Also, use 80% of the data for training.


In [14]:
set.seed(1234)

noaa_split <- noaa_subsetwranglednn %>% initial_split(prop = 0.8) # splitting the dataset
noaa_training <- training(noaa_split) # creating the training subset
noaa_testing <- testing(noaa_split) # creating the testing subset

SyntaxError: invalid syntax (4138142139.py, line 3)

Next, looking at just the **training set**, plot histograms or box plots of the variables (`relative_humidity`, `dry_bulb_temp_f`, `precip`, `wind_speed`,  `station_pressure`) for an intial look of their distributions using `tidyverse`'s `ggplot`. Leave the testing set as is because it is good practice to not see the testing set until evaluating the final model.


In [15]:
install.packages("ggplot2") # installing the ggplot package
library(ggplot2)

ggplot(noaa_training, mapping = aes(x = noaa_training$relative_humidity)) +
  geom_histogram(color = "black", fill = "red")

ggplot(noaa_training, mapping = aes(x = noaa_training$dry_bulb_temp_f)) +
  geom_histogram(color = "black", fill = "red")

ggplot(noaa_training, mapping = aes(x = noaa_training$precip)) +
  geom_histogram(color = "black", fill = "red")

ggplot(noaa_training, mapping = aes(x = noaa_training$wind_speed)) +
  geom_histogram(color = "black", fill = "red")

ggplot(noaa_training, mapping = aes(x = noaa_training$station_pressure)) +
  geom_histogram(color = "black", fill = "red")

SyntaxError: invalid syntax (2384042941.py, line 4)

<a id="cell8"></a>
## 8. Linear Regression 
After exploring the dataset more, you are now ready to start creating models to predict the precipitation (`precip`).

Create simple linear regression models where `precip` is the response variable and each of `relative_humidity`, `dry_bulb_temp_f`,`wind_speed` or `station_pressure` will be a predictor variable, e.g. `precip ~ relative_humidity`, `precip ~ dry_bulb_temp_f`, etc. for a total of four simple models. 
Additionally, visualize each simple model with a scatter plot.


In [16]:
attach(noaa_training) # to create separate elements for each column (variable)

model1 <- lm(noaa_training$precip~relative_humidity)
summary(model1)
  
ggplot(noaa_training, mapping = aes(x = relative_humidity, y = noaa_training$precip)) +
  geom_point() +
  geom_smooth(method = "lm", formula = y~x, 
              col = "green", se = FALSE, na.rm = TRUE)

SyntaxError: invalid syntax (404497712.py, line 3)

In [17]:
model2 <- lm(noaa_training$precip~dry_bulb_temp_f)
summary(model2)

ggplot(noaa_training, mapping = aes(x = dry_bulb_temp_f, y = noaa_training$precip)) +
  geom_point() +
  geom_smooth(method = "lm", formula = y~x, 
              col = "blue", se = FALSE, na.rm = TRUE)

SyntaxError: invalid syntax (3526673494.py, line 1)

In [18]:
model3 <- lm(noaa_training$precip~wind_speed)
summary(model3)

ggplot(noaa_training, mapping = aes(x = wind_speed, y = noaa_training$precip)) +
  geom_point() +
  geom_smooth(method = "lm", formula = y~x, 
              col = "red", se = FALSE, na.rm = TRUE)

SyntaxError: invalid syntax (960578477.py, line 1)

In [19]:
model4 <- lm(noaa_training$precip~station_pressure)
summary(model4)

ggplot(noaa_training, mapping = aes(x = station_pressure, y = noaa_training$precip)) +
  geom_point() +
  geom_smooth(method = "lm", formula = y~x, 
              col = "purple", se = FALSE, na.rm = TRUE)

SyntaxError: invalid syntax (1504376728.py, line 1)

<a id="cell9"></a>
## 9. Improve the Model
Now, try improving the simple models you created in the previous section. 

Create at least two more models, each model should use at least one of the different techniques:
1. Add more features/predictors
2. Add regularization (L1, L2 or a mix)
3. Add a polynomial component

Also, for each of the models you create, check the model performance using the **training set** and a metric like MSE, RMSE, or R-squared.

Consider using `tidymodels` if you choose to add regularization and tune lambda.


In [20]:
## Improved model 1 (using all 4 explanatory variables, in a ridge model, also pbtaining lambda by grid search):

# for computing such a model, the NA (missing values should be deleted from the training dataset)
map(noaa_training, ~sum(is.na(.))) # how many missing values do we have in each column of the dataset
noaa_training1 <- noaa_training %>% drop_na(c(relative_humidity, dry_bulb_temp_f, wind_speed, station_pressure, precip))
map(noaa_training1, ~sum(is.na(.))) # we have now deleted ALL the missing values (NAs)

dim(noaa_training)
dim(noaa_training1) # as a result of dropping the NAs, we will inevitably lose about 1000 observations

improved_recipe <- recipe(precip ~ relative_humidity + 
                            dry_bulb_temp_f + wind_speed + station_pressure, 
                          data = noaa_training1)

improved_ridge <- linear_reg(penalty = tune(), mixture = 0) %>%
  set_engine("glmnet")

noaa_cvfolds <- vfold_cv(noaa_training1)

lambda_grid <- grid_regular(levels = 250,
                            penalty(range = c(-3,0.3)))

ridge_wf <- workflow() %>%
  add_recipe(improved_recipe)

ridge_grid <- tune_grid(
  
  ridge_wf %>% add_model(improved_ridge),
  resamples = noaa_cvfolds,
  grid = lambda_grid
) 

show_best(ridge_grid, metric = "rmse") # The best penalty (with the lowest RMSE) is found as 0.00749

# This lambda now will be used in the ridge regression:
improved_recipe <- recipe(precip ~ relative_humidity + 
                            dry_bulb_temp_f + wind_speed + station_pressure, 
                          data = noaa_training1)

improved_ridge <- linear_reg(penalty = 0.00749, mixture = 0) %>%
  set_engine("glmnet")

ridge_workflow <- workflow() %>%
  add_recipe(improved_recipe)

ridge_fit <- ridge_workflow %>%
  add_model(improved_ridge) %>%
  fit(data = noaa_training1) 

ridge_fit %>% 
  pull_workflow_fit() %>%
  tidy() # the coefficients of the ridge regression

# I think in this case, the RMSE is going to be the same as the mean of RMSE
# as computed in the Grid Search step.
# That gave us the optimum lambda. So I would say RMSE is equal to 0.0382.
ridge_rmse_training <- 0.0382


## IMPROVED MODEL 2 (A simple polynomial model by using the second degree polynomial of explanatory variables of relative humidity and station pressure):

relative_humidity_sq <- relative_humidity^2 
station_pressure_sq <- station_pressure^2
improved_polynomial <- lm(noaa_training$precip ~ relative_humidity + relative_humidity_sq + station_pressure + station_pressure_sq)
summary(improved_polynomial) # the results

poly_rmse_training <- improved_polynomial$residuals^2 %>% mean() %>% sqrt() # the RMSE of the model

SyntaxError: invalid syntax (76833021.py, line 4)

<a id="cell10"></a>
## 10. Find Best Model
Compare the regression metrics of each model from section 9 to find the best model overall. To do this, 

1. Evaluate the models on the **testing set** using at least one metric (like MSE, RMSE or R-squared).
2. After calculating the metrics on the testing set for each model, print them out in as a table to easily compare. You can use something like:
```
model_names <- c("model_1", "model_2", "model_3")
train_error <- c("model_1_value", "model_2_value", "model_3_value")
test_error <- c("model_1_value", "model_2_value", "model_3_value")
comparison_df <- data.frame(model_names, train_error, test_error)
```
3. Finally, from the comparison table you create, conclude which model performed the best.


In [1]:
# Fitting the models on the testing data:
## A. Model 1:
test_results_m1 <- ridge_fit %>% # calling out the data for the first model
  predict(new_data = noaa_testing) %>% # predicting precipitation (dependent variable) in our model by using the testing data on explanatory variables
  mutate(truth = noaa_testing$precip) # adding the corresponding actual precipitation to the table

ridge_rmse_testing <- rmse(test_results_m1, truth = truth, estimate = .pred)[,3] %>% as.numeric()

## B. Model 2:

# First we need to fit the model based on the training data:

# Defining the engine behind the model
poly_spec <- linear_reg() %>% 
  set_engine("lm")

# Training the model
poly_fit <- poly_spec %>%
  fit(noaa_training$precip ~ poly(relative_humidity, degree = 2, raw = TRUE) +
        poly(station_pressure, degree = 2, raw = TRUE)
      , data = noaa_training)

# Using the testing data on the trained model:
test_results_m2 <- poly_fit %>%
  predict(new_data = noaa_testing) %>%
  mutate(truth = noaa_testing$precip)

poly_rmse_testing <- rmse(test_results_m2, truth = truth, estimate = .pred)[,3] %>% as.numeric()

# Comparing the two models by using a table (data frame):
model_names <- c("Ridge", "Poly")
train_rmse <- c(ridge_rmse_training, poly_rmse_training)
test_rmse <- c(ridge_rmse_testing, poly_rmse_testing)

comparison_df <- data.frame(model_names, train_rmse, test_rmse)
comparison_df

# It is interesting that the training error, as measured by RMSE, was lower for the Ridge model,
# fitting the testing data on the model would result in a lower RMSE for the poly model. 
# In this sense, although the Ridge model is better in explaining the variation of the data,
# it seems like it is relatively over-fitted. To the degree that the predictable ability of the poly model is 
# higher than the ridge model.


SyntaxError: invalid syntax (153721724.py, line 3)

## Author(s)

<h4> Yiwen Li </h4>

## Contributions

<h4> Tiffany Zhu </h4>

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
